In [1]:
#pip install datasets


In [2]:
import os
import pandas as pd
import json
import csv
from tqdm import tqdm
import numpy as np

In [3]:
from document_preprocessor import RegexTokenizer
from indexing import Indexer, IndexType
from ranker import *
import pandas as pd
from tqdm import tqdm
from relevance import *
from collections import defaultdict
import csv
import gzip
import jsonlines
import json
from collections import defaultdict
import os
from tqdm import tqdm

/opt/anaconda3/envs/si650/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# csv_file_path = 'documents_full_contents - full_documents.csv'
# jsonl_file_path = 'data/documents_full_contents.jsonl'

# with open(csv_file_path, 'r', newline='', encoding='utf-8') as csv_file:
#     reader = csv.DictReader(csv_file)
#     with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
#         for row in reader:
#             jsonl_file.write(json.dumps(row) + '\n')


In [ ]:
data.head()

,docid,content
0,99,"What is your favourite non-English saying, and..."
1,240,What would you do if someone offers u a free a...
2,409,"To all airplane riders, what is the most scare..."
3,461,"What lie do you tell routinely? Why? ""My favor..."
4,626,What is a history fact that is so stupid it do...


In [ ]:
np.random.seed(42)

In [ ]:
from datasets import load_dataset

# Load the dataset
posts = load_dataset('SocialGrep/ten-million-reddit-answers', 'posts')

# Print the path to the dataset files or display a sample


/opt/anaconda3/envs/si650/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
len(posts)

1

In [ ]:
# Load the dataset
comments = load_dataset('SocialGrep/ten-million-reddit-answers', 'comments')

In [ ]:
comments = comments['train'].to_pandas()

In [ ]:
len(comments)

10000000

In [ ]:
#comments.to_csv("reddit_comments.csv", index=False)

In [ ]:
posts = posts['train'].to_pandas()

In [ ]:
posts

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,hp5581,2qh1i,askreddit,False,2020-07-11 06:10:09+00:00,https://old.reddit.com/r/AskReddit/comments/hp...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/hp...,,What is something that you will never forget ?,6
1,post,jkh2hg,2qh1i,askreddit,False,2020-10-29 19:40:28+00:00,https://old.reddit.com/r/AskReddit/comments/jk...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jk...,,What one moment made someone nose dive their e...,2
2,post,jnk1pq,2qh1i,askreddit,False,2020-11-03 22:08:30+00:00,https://old.reddit.com/r/AskReddit/comments/jn...,self.askreddit,,[deleted],"What is the absolute greatest, pinnacle, best ...",2
3,post,jepd7w,2qh1i,askreddit,False,2020-10-20 13:22:24+00:00,https://old.reddit.com/r/AskReddit/comments/je...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/je...,,What is some horrible advice that you have bee...,7
4,post,jw2fs9,2qh1i,askreddit,False,2020-11-17 21:53:04+00:00,https://old.reddit.com/r/AskReddit/comments/jw...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jw...,,What item you wanted so bad when you were a ki...,2
...,...,...,...,...,...,...,...,...,...,...,...,...
631058,post,k3j0zv,2qh1i,askreddit,False,2020-11-29 22:38:54+00:00,https://old.reddit.com/r/AskReddit/comments/k3...,self.askreddit,,[deleted],"In Antarctica, does it get warmer instead of c...",2
631059,post,jm6zm1,2qh1i,askreddit,False,2020-11-01 18:16:05+00:00,https://old.reddit.com/r/AskReddit/comments/jm...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jm...,,What is it like to be blind?,2
631060,post,joj1cw,2qh1i,askreddit,False,2020-11-05 13:48:43+00:00,https://old.reddit.com/r/AskReddit/comments/jo...,self.askreddit,,[removed],What is the reporting % when you google electi...,1
631061,post,jdkwa8,2qh1i,askreddit,False,2020-10-18 17:59:24+00:00,https://old.reddit.com/r/AskReddit/comments/jd...,self.askreddit,,[deleted],"What was your dream job as a kid, your job now...",1


In [ ]:
comments

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,body,sentiment,score
0,1,ge7etc6,2qh1i,askreddit,False,2020-11-30 23:59:59+00:00,https://old.reddit.com/r/AskReddit/comments/k3...,"The Good, The Bad, and the Ugly, it’s just so ...",-0.3400,1
1,1,ge7etdb,2qh1i,askreddit,False,2020-11-30 23:59:59+00:00,https://old.reddit.com/r/AskReddit/comments/k3...,"Well, as a Bohemian, it really just doesn't ma...",0.1931,1
2,1,ge7et9h,2qh1i,askreddit,False,2020-11-30 23:59:58+00:00,https://old.reddit.com/r/AskReddit/comments/k4...,Telekinesis,NaN,2
3,1,ge7eta2,2qh1i,askreddit,False,2020-11-30 23:59:58+00:00,https://old.reddit.com/r/AskReddit/comments/k4...,Glitter,NaN,2
4,1,ge7et8f,2qh1i,askreddit,False,2020-11-30 23:59:57+00:00,https://old.reddit.com/r/AskReddit/comments/k4...,Abs.,NaN,1
...,...,...,...,...,...,...,...,...,...,...
9999995,1,g89mu2k,2qh1i,askreddit,False,2020-10-09 23:53:53+00:00,https://old.reddit.com/r/AskReddit/comments/j7...,[deleted],NaN,1
9999996,1,g89mtzn,2qh1i,askreddit,False,2020-10-09 23:53:52+00:00,https://old.reddit.com/r/AskReddit/comments/j8...,Where is the pen they keep the politicians in?,0.0000,1
9999997,1,g89mu0q,2qh1i,askreddit,False,2020-10-09 23:53:52+00:00,https://old.reddit.com/r/AskReddit/comments/j8...,Steve Wozniak,NaN,2
9999998,1,g89mu13,2qh1i,askreddit,False,2020-10-09 23:53:52+00:00,https://old.reddit.com/r/AskReddit/comments/j8...,but fuckinnn what u got a boyfriend stylll fam...,0.5413,2


In [ ]:
posts.head()

,type,id,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,permalink,domain,url,selftext,title,score
0,post,hp5581,2qh1i,askreddit,False,2020-07-11 06:10:09+00:00,https://old.reddit.com/r/AskReddit/comments/hp...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/hp...,,What is something that you will never forget ?,6
1,post,jkh2hg,2qh1i,askreddit,False,2020-10-29 19:40:28+00:00,https://old.reddit.com/r/AskReddit/comments/jk...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jk...,,What one moment made someone nose dive their e...,2
2,post,jnk1pq,2qh1i,askreddit,False,2020-11-03 22:08:30+00:00,https://old.reddit.com/r/AskReddit/comments/jn...,self.askreddit,,[deleted],"What is the absolute greatest, pinnacle, best ...",2
3,post,jepd7w,2qh1i,askreddit,False,2020-10-20 13:22:24+00:00,https://old.reddit.com/r/AskReddit/comments/je...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/je...,,What is some horrible advice that you have bee...,7
4,post,jw2fs9,2qh1i,askreddit,False,2020-11-17 21:53:04+00:00,https://old.reddit.com/r/AskReddit/comments/jw...,self.askreddit,https://www.reddit.com/r/AskReddit/comments/jw...,,What item you wanted so bad when you were a ki...,2


In [ ]:
#comments_df = pd.read_csv('reddit_comments.csv')

In [ ]:
#len(comments_df)

In [ ]:
# posts.to_csv("reddit_posts.csv", index=False)

In [ ]:
# posts = pd.read_csv("reddit_posts.csv")

# comments = pd.read_csv("reddit_comments.csv")

In [ ]:
#len(posts)

In [ ]:
#len(comments)

<h2>Merge datasets</h2>

In [ ]:
posts['Post_created_date'] = pd.to_datetime(posts['created_utc'], unit = 's')

In [ ]:
comments['post permalink'] = comments['permalink'].apply(lambda s: "/".join(s.split("/")[:-2]) + "/")

In [ ]:
#prepare data
posts_subset = posts[['title', 'permalink', 'Post_created_date']]

comments = comments.drop(columns = ['permalink'])

merge = posts_subset.merge(comments, how = 'left', left_on = 'permalink', right_on = 'post permalink')

## saving data 

In [ ]:
# new numeric index 
merge = merge.reset_index()
merge = merge.drop(columns = ["id"])
merge = merge.rename(columns = {'index': 'docid'})

In [ ]:
merge = merge.sort_values(by = 'created_utc', ascending = True)

In [ ]:
#merge.to_csv("data/redditQandA.csv", index = False)

In [ ]:
titles = list(merge['title'].astype(str).values)
comments = list(merge['body'].astype(str).values)
idx = list(merge['docid'].values)

In [ ]:
used_idx = np.random.choice(idx, size = 200000, replace = False)

In [ ]:
selected_data = merge[merge['docid'].isin(used_idx)]

In [ ]:
len(selected_data)

200000

In [ ]:
selected_data

,docid,title,permalink,Post_created_date,type,subreddit.id,subreddit.name,subreddit.nsfw,created_utc,body,sentiment,score,post permalink
9917577,9917577,How do you feel ?,https://old.reddit.com/r/AskReddit/comments/j8...,2020-10-09 23:52:44+00:00,1.0,2qh1i,askreddit,False,2020-10-09 23:53:59+00:00,Horny,NaN,2.0,https://old.reddit.com/r/AskReddit/comments/j8...
3955740,3955740,What is the best deleted scene from a movie or...,https://old.reddit.com/r/AskReddit/comments/j8...,2020-10-09 23:53:08+00:00,1.0,2qh1i,askreddit,False,2020-10-09 23:53:59+00:00,the rape scene from the nostalgia critic movie,-0.7783,0.0,https://old.reddit.com/r/AskReddit/comments/j8...
9177443,9177443,"What do you believe, but cannot prove?",https://old.reddit.com/r/AskReddit/comments/j8...,2020-10-09 19:08:01+00:00,1.0,2qh1i,askreddit,False,2020-10-09 23:54:18+00:00,I don't look anything like my grandfather thou...,-0.5439,5.0,https://old.reddit.com/r/AskReddit/comments/j8...
2187082,2187082,What do you need the most right now?,https://old.reddit.com/r/AskReddit/comments/j8...,2020-10-09 23:47:18+00:00,1.0,2qh1i,askreddit,False,2020-10-09 23:54:58+00:00,Same,NaN,2.0,https://old.reddit.com/r/AskReddit/comments/j8...
8636117,8636117,People often say that vegetables taste bad. Ho...,https://old.reddit.com/r/AskReddit/comments/j8...,2020-10-09 23:51:02+00:00,1.0,2qh1i,askreddit,False,2020-10-09 23:55:19+00:00,Who even thinks this? Most people like veggies...,0.4173,3.0,https://old.reddit.com/r/AskReddit/comments/j8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5569606,5569606,"What book, in your opinion, is a “must read”?",https://old.reddit.com/r/AskReddit/comments/k3...,2020-11-30 13:24:27+00:00,1.0,2qh1i,askreddit,False,2020-11-30 23:59:17+00:00,"""The Hobbit"" also, ""Jane Eyre"" and ""The Bone C...",0.0000,1.0,https://old.reddit.com/r/AskReddit/comments/k3...
5905230,5905230,People who wear shorts and sandals in (32 F or...,https://old.reddit.com/r/AskReddit/comments/k4...,2020-11-30 23:08:56+00:00,1.0,2qh1i,askreddit,False,2020-11-30 23:59:32+00:00,I'm from the north and lived in Alaska. When y...,-0.8360,1.0,https://old.reddit.com/r/AskReddit/comments/k4...
7944882,7944882,"With the last month starting, what is going to...",https://old.reddit.com/r/AskReddit/comments/k4...,2020-11-30 23:55:35+00:00,1.0,2qh1i,askreddit,False,2020-11-30 23:59:33+00:00,World War V.,-0.5994,1.0,https://old.reddit.com/r/AskReddit/comments/k4...
3312929,3312929,Someone offers you $100/hr to watch any movie ...,https://old.reddit.com/r/AskReddit/comments/k3...,2020-11-30 13:49:54+00:00,1.0,2qh1i,askreddit,False,2020-11-30 23:59:44+00:00,"school of rock, easy",0.4404,1.0,https://old.reddit.com/r/AskReddit/comments/k3...


In [ ]:
data = selected_data[['docid','title']]

In [ ]:
# idx_to_post = {}

# for i in tqdm(range(len(used_idx))):
#     docid = idx[i]
#     title = titles[i]
#     comment = comments[i]
#     if '[deleted]' in comment or '[removed]' in comment or '**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEFORE TAKING ACTION.*' in comment: 
#         continue
#     if id not in list(idx_to_post.values()):
#         idx_to_post[docid] = title + " " + comment + " "
#         continue
#     curr = idx_to_post[docid]
#     if len(curr) <= 30000:
#         idx_to_post[docid] = idx_to_post[docid] + comment + " "

100%|██████████| 200000/200000 [10:14<00:00, 325.35it/s] 


In [ ]:
idx_to_post

{7188056: 'Every time someone lies to you, $100 gets deposited to your bank account. What is the fastest way for you to get rich? Where is the pen they keep the politicians in? ',
 5710468: "What are the most race suggestive names (along with the race they suggest)? D'Quanda ",
 3399481: 'What billionaire do you think would be the coolest to hang out with and why? Steve Wozniak ',
 2538133: 'Women of Reddit, would you have sex with Manuel? And why did you comment yes? but fuckinnn what u got a boyfriend stylll fam? cah like manz on the block ygm g mans around like mans moving bare like cant chat too loose on this styll feds always watching but fuckinnn meet me farm block ygm anyway fuckkkk campolliiiiii ',
 2191117: "What is a tune everybody knows, but nobody knows where it's from? [Herb Alpert and the Tijuana Brass - Spanish Flea](https://www.youtube.com/watch?v=hxsOXOPni0o) ",
 2883281: 'What are you shamefully bad at learning? Everything ',
 9917579: 'How do you feel ? With my hands

In [ ]:
id

<function id(obj, /)>

In [ ]:
# with open("selected_200k.json", "w") as f:
#     for k, v in list(idx_to_post.items()):
#         k = int(k)
#         d = {'docid': k, 'text': v}
#         print(json.dumps(d), file = f)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json("selected_200k.json", lines=True)


In [ ]:
df.to_csv("selected_200k.csv", index = False)

In [14]:
# 1minute:4 secs
from indexing import BasicInvertedIndex, Indexer, IndexType
from document_preprocessor import RegexTokenizer
from ranker import BM25


with open('../data/stopwords.txt', 'r') as f:
    stopwords = set(f.read().splitlines())
DATASET_PATH = '../data/documents_full_contents.jsonl'

# docid_to_network_features = {}
max_docs=20000

document_index=Indexer.create_index(
            IndexType.BasicInvertedIndex, DATASET_PATH , RegexTokenizer('\w+'),
            stopwords, 5, text_key='content', max_docs=max_docs
        )

document_preprocessor = RegexTokenizer() 
bm25_scorer = BM25(document_index, parameters={'b': 0.75, 'k1': 1.2, 'k3': 8})

ranker = Ranker(
    index=document_index,
    document_preprocessor=document_preprocessor,
    stopwords=stopwords,
    scorer=bm25_scorer,
   
)



20000it [00:00, 43265.23it/s]


In [7]:
len(document_index.index.keys())

6512

In [ ]:
import sys
import re

In [12]:
df = pd.read_csv('../data/Rahat_BM25.xlsx - merged_BM25.csv')

In [ ]:
df.rel.isna().sum()

0

In [15]:
relevance_data_filename = '../data/Rahat_BM25.xlsx - merged_BM25.csv'

relevance_scores = run_relevance_tests(relevance_data_filename, bm25_scorer)

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

AttributeError: 'BM25' object has no attribute 'query'

In [11]:
relevance_data_filename 

'../data/Rahat_BM25.xlsx - merged_BM25.csv'